## Importación

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import classification_report
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTEN

In [2]:
df=pd.read_csv("https://raw.githubusercontent.com/nathygonzalez/PR06-Machine-Learning-Group-Project/develop/data/df_cleaned.csv")

C:\Users\carlo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (99) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## depuración columna "gk"

In [3]:
df['gk'] =df['gk'].replace("+","")

In [4]:
df['gk'] = df['gk'].str.replace(r'\+', '', regex=True)


In [5]:
df.gk.fillna(0,inplace=True)

In [6]:
df['gk']=df.gk.astype(int)

In [7]:
df['gk']

0        15
1        16
2        14
3        17
4        87
         ..
87742     0
87743     0
87744     0
87745     0
87746     0
Name: gk, Length: 87747, dtype: int32

## Añadir variable años pendientes de contrato

In [8]:
df['años_de_contrato']=np.where((df['club_contract_valid_until']-df['año_version'])<0,0,(df['club_contract_valid_until']-df['año_version']))

In [9]:
#comprobación
df.groupby('años_de_contrato')['años_de_contrato'].count()

años_de_contrato
0.0    30707
1.0    21182
2.0    17779
3.0     9287
4.0     4872
5.0     2502
6.0     1413
7.0        3
8.0        2
Name: años_de_contrato, dtype: int64

## Eliminación de ruido

In [10]:
#elimino columnas (según Sergio)
'''df.drop(['sofifa_id',
 'short_name',
 'long_name',
 'player_positions',
     'dob', 'club_team_id',
 'club_name',
 'league_name',
 'league_level',
'club_jersey_number',
 'club_joined',
 'club_contract_valid_until',
 'nationality_id',
 'nationality_name',
 'nation_team_id','work_rate',
 'body_type',
 'real_face',
    'player_traits'],axis=1,inplace=True)'''

"df.drop(['sofifa_id',\n 'short_name',\n 'long_name',\n 'player_positions',\n     'dob', 'club_team_id',\n 'club_name',\n 'league_name',\n 'league_level',\n'club_jersey_number',\n 'club_joined',\n 'club_contract_valid_until',\n 'nationality_id',\n 'nationality_name',\n 'nation_team_id','work_rate',\n 'body_type',\n 'real_face',\n    'player_traits'],axis=1,inplace=True)"

In [11]:
#eliminamos columnas que sabemos seguro que sólo aportan ruido (según Carlos)
df.drop(["sofifa_id",'short_name', 'long_name',"nationality_id","dob",'club_team_id', 'club_name', 'league_name',
        'club_jersey_number', 'club_joined', 'club_contract_valid_until','nationality_id', 'nationality_name','player_traits',
        'ranking','puntos','año_version'],axis=1,inplace=True)

## Agrupar columna bodytype

In [12]:
Lean=['Lean (170-)','Lean (185+)','Lean (170-185)']
Normal=['Normal (170-)','Normal (170-185)','Normal (185+)']
Stocky=['Stocky (170-)','Stocky (170-185)','Stocky (185+)']

#agrupo posiciones en posiciones genéricas    
for i in df.index:
    if df.at[i,'body_type'] in Lean:
        df.at[i,'body_type']='Lean'
    elif df.at[i,'body_type'] in Normal:
        df.at[i,'body_type']='Normal'
    elif df.at[i,'body_type'] in Stocky:
        df.at[i,'body_type']='Stocky'
    else:
        pass

## Exploración feature engineering

In [13]:
df.preferred_foot.value_counts() #categorica a OHE

Right    66970
Left     20777
Name: preferred_foot, dtype: int64

In [14]:
df.work_rate.value_counts() #categorica transformarla con un ordinal encoder

Medium/Medium    47248
High/Medium      15911
Medium/High       8196
High/High         4712
Medium/Low        4072
High/Low          3332
Low/Medium        2078
Low/High          2058
Low/Low            140
Name: work_rate, dtype: int64

In [15]:
df.body_type.value_counts() #categorica a OHE

Normal    51581
Lean      30898
Stocky     5099
Unique      169
Name: body_type, dtype: int64

In [16]:
df.real_face.value_counts() #categorica a OHE

No     78885
Yes     8862
Name: real_face, dtype: int64

## Feature engineering (transformar todas las variables a numéricas menos player_positions)

In [17]:
#preparación para transformación categorica a num
work_rate_ord=['Low/Low','Low/Medium','Low/High','Medium/Low','Medium/Medium','Medium/High','High/Low','High/Medium','High/High']
league_level_ord=['0.0','4.0','3.0','2.0','1.0']
df['league_level']=df['league_level'].apply(str)

In [18]:
#Ordinal Encoder
df[['work_rate','league_level']]=OrdinalEncoder(categories=[work_rate_ord,league_level_ord]).fit_transform(df[['work_rate','league_level']])

In [19]:
#OHE
df[['preferred_foot','body_type_1','body_type_2','body_type_3','real_face']]=OneHotEncoder(drop='first',sparse=False).fit_transform(df[['preferred_foot','body_type','real_face']])                                          

In [20]:
#comprobar columnas
categorical = df.dtypes == object
categorical[categorical==True]

player_positions    True
body_type           True
dtype: bool

In [21]:
#eliminar columna body_type
df.drop(["body_type"],axis=1,inplace=True)

## Filtramos el dataset por posicion

In [22]:
#filtramos dataset para evitar excesiva coliniaridad
df=df[df['player_positions']=='centrocampista']
#eliminar columna player_positions
df.drop(["player_positions"],axis=1,inplace=True)

In [23]:
df.head()

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_level,club_position,nation_team_id,...,gk,progresion_anual,overall_team/year,teams_in_league/year,mean_league/year,mean_team/year,años_de_contrato,body_type_1,body_type_2,body_type_3
1,92,92,79000000.0,375000.0,29,185,80,4.0,1,1,...,16,0.010870,2418.0,20.0,71.357143,75.562500,3.0,1.0,0.0,0.0
2,90,90,54500000.0,275000.0,30,180,80,4.0,0,1,...,14,0.000000,2061.0,18.0,71.280514,79.269231,2.0,1.0,0.0,0.0
5,89,89,36000000.0,250000.0,30,170,65,4.0,1,1,...,14,-0.011236,1917.0,20.0,71.357143,79.875000,3.0,1.0,0.0,0.0
8,88,88,39000000.0,200000.0,29,183,79,4.0,0,1,...,18,-0.022727,2061.0,18.0,71.280514,79.269231,1.0,1.0,0.0,0.0
9,88,88,33000000.0,200000.0,31,170,72,4.0,0,0,...,14,-0.011364,2061.0,18.0,71.280514,79.269231,2.0,1.0,0.0,0.0


## Features selection

In [24]:
target=[]
#elegimos el % de target para que sea 0 y 1
for i in df.progresion_anual:
    if i>0.1:
        target.append(1)
    else:
        target.append(0)
df["target"]=target

In [25]:
X=df.drop(["progresion_anual","target"],axis=1)
y=df.target

In [31]:
#seleccion de columnas 
sel = SelectFromModel(RandomForestClassifier(n_estimators = 200))
sel.fit(X,y)

SelectFromModel(estimator=RandomForestClassifier(n_estimators=200))

In [32]:
#columnas que coge
sel.get_support()

array([ True,  True,  True, False,  True,  True,  True, False, False,
       False, False, False, False, False, False, False, False,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False, False,
       False, False,  True, False, False, False, False, False,  True,
        True,  True, False,  True, False,  True,  True, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False,  True,  True,  True,  True, False,
       False, False, False])

In [33]:
selected_feat= X.columns[(sel.get_support())]
len(selected_feat) #selecciona 51 columnas

52

In [34]:
#vemos que variables son
selected_feat

Index(['overall', 'potential', 'value_eur', 'age', 'height_cm', 'weight_kg',
       'pace', 'passing', 'dribbling', 'defending', 'physic',
       'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
       'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
       'movement_reactions', 'movement_balance', 'power_shot_power',
       'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
       'mentality_aggression', 'mentality_interceptions',
       'mentality_positioning', 'mentality_vision', 'mentality_penalties',
       'mentality_composure', 'defending_marking_awareness',
       'defending_standing_tackle', 'defending_sliding_tackle',
       'goalkeeping_reflexes', 'lf', 'cf', 'rf', 'lam', 'ram', 'lm', 'rm',
       'overall_team/year', 'teams_in_league/year', '

In [43]:
#redefinimos X_eliminando las que no tienen sentido:
list=selected_feat.to_list()
list.remove('goalkeeping_reflexes')
X=df[list]

## Primer modelo naive

In [44]:
#split el dataframe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [45]:
model=RandomForestClassifier()

In [46]:
model.fit(X_train,y_train)

RandomForestClassifier()

In [47]:
model.score(X_test,y_test)

0.95442690459849

In [48]:
y_pred=model.predict(X_test)

In [49]:
print(classification_report(y_test,y_pred)) 
#Se observa accury muy alta => consecuencia posible de df muy desbalanceado

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6958
           1       0.35      0.02      0.03       327

    accuracy                           0.95      7285
   macro avg       0.65      0.51      0.51      7285
weighted avg       0.93      0.95      0.93      7285



In [50]:
#desbalanceo de X
6958/7285

0.9551132463967056

## Segundo modelo con undersampling

In [51]:
rus = RandomUnderSampler(random_state=0)
X_res, y_res =rus.fit_resample(X, y)

#split el dataframe
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=0)



In [52]:
model=RandomForestClassifier()
model.fit(X_train,y_train)

y_pred=model.predict(X_test)

print(classification_report(y_test, y_pred))
#random forest classifier consigue un 81% de accuracy

              precision    recall  f1-score   support

           0       0.82      0.79      0.81       360
           1       0.80      0.83      0.82       367

    accuracy                           0.81       727
   macro avg       0.81      0.81      0.81       727
weighted avg       0.81      0.81      0.81       727



In [53]:
model2=SVC()
model2.fit(X_train,y_train)
y_pred=model2.predict(X_test)
print(classification_report(y_test, y_pred))
#SVC consigue un 71% de accuracy

              precision    recall  f1-score   support

           0       0.76      0.62      0.68       360
           1       0.68      0.80      0.74       367

    accuracy                           0.71       727
   macro avg       0.72      0.71      0.71       727
weighted avg       0.72      0.71      0.71       727



## Tercer modelo con Over-sampling

In [54]:
res= SMOTEN()

In [55]:
X_res,y_res=res.fit_resample(X, y)

In [56]:
#comprobación
sum(y_res)/len(y_res) 

0.5

In [57]:
#split el dataframe
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=0)

In [58]:
model3=RandomForestClassifier()
model3.fit(X_train,y_train)
y_pred=model3.predict(X_test)
print(classification_report(y_test, y_pred))
#random forest classifier con oversampling consigue un 97% de accuracy

              precision    recall  f1-score   support

           0       0.96      0.99      0.97      6814
           1       0.99      0.96      0.97      7030

    accuracy                           0.97     13844
   macro avg       0.97      0.97      0.97     13844
weighted avg       0.97      0.97      0.97     13844



In [59]:
model4=SVC()
model4.fit(X_train,y_train).score(X_test,y_test)
y_pred=model4.predict(X_test)
print(classification_report(y_test, y_pred))
#SVC con oversampling consigue un 78% de accuracy

              precision    recall  f1-score   support

           0       0.81      0.72      0.77      6814
           1       0.76      0.84      0.80      7030

    accuracy                           0.78     13844
   macro avg       0.79      0.78      0.78     13844
weighted avg       0.79      0.78      0.78     13844



## Parameter tunnig

In [125]:
param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

In [127]:
%%time
from sklearn.model_selection import GridSearchCV
CV_rfc = GridSearchCV(estimator=model3, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)

Wall time: 18min 10s


GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 5, 6, 7, 8],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [200, 500]})

In [128]:
CV_rfc.best_params_

{'criterion': 'entropy',
 'max_depth': 8,
 'max_features': 'auto',
 'n_estimators': 500}

In [60]:
model_def=RandomForestClassifier(max_features='auto', n_estimators= 500, max_depth=8, criterion='entropy')
model_def.fit(X_train,y_train)
y_pred=model_def.predict(X_test)
print(classification_report(y_test, y_pred))
#random forest classifier con oversampling consigue un 95% de accuracy

              precision    recall  f1-score   support

           0       0.97      0.93      0.95      6814
           1       0.93      0.97      0.95      7030

    accuracy                           0.95     13844
   macro avg       0.95      0.95      0.95     13844
weighted avg       0.95      0.95      0.95     13844



## Mirar la previsión de jugadores de fifa22

In [95]:
df22=pd.read_csv('data/players_22.csv')
#eliminamos columnas de url que no aportan información
df22 = df22[df22.columns.drop(df22.filter(regex='url').columns)]

C:\Users\carlo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (25,108) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [96]:
#funcion transformación en lista
def lista_posicion(x):
    return (x.replace(' ','').split(','))

In [97]:
#transforma valores de columna player_positions en una lista
df22['player_positions']=df22['player_positions'].apply(lambda x: lista_posicion(x))

In [98]:
#defino listas para agrupar posiciones por 4 categorias
pos_defensas=['LCB','RCB','CB','RB','LB','RWB','LWB']
pos_medios=['LW','LCM','RCM','CAM','RW','LDM','CDM','CM','RDM','RM','RAM','LAM','LM']
pos_delanteros=['CF','ST','RS','LS','RF']

In [99]:
#agrupo posiciones en posiciones genéricas    
for i in df22.index:
            if len(df22.at[i,'player_positions'])==1:
                if df22.at[i,'player_positions'][0]=='GK':
                    df22.at[i,'player_positions']='portero'
                if df22.at[i,'player_positions'][0] in pos_defensas:
                    df22.at[i,'player_positions']='defensa'
                elif df22.at[i,'player_positions'][0] in pos_medios:
                    df22.at[i,'player_positions']='centrocampista'
                elif df22.at[i,'player_positions'][0] in pos_delanteros:
                    df22.at[i,'player_positions']='delantero'
            else:
                if df22.at[i,'player_positions'][0] in pos_defensas:
                    df22.at[i,'player_positions']='defensa'
                elif df22.at[i,'player_positions'][0] in pos_medios:
                    df22.at[i,'player_positions']='centrocampista'
                elif df22.at[i,'player_positions'][0] in pos_delanteros:
                    df22.at[i,'player_positions']='delantero'
                elif df22.at[i,'player_positions'][0]=='GK':
                    df22.at[i,'player_positions']='portero'

In [100]:
#filtramos dataset 
df22=df22[df22['player_positions']=='centrocampista']

In [101]:
#Agrupa por Equipo y año versión(temporada) y añade el total en cada jugador
df22['overall_team/year'] = df22.groupby('club_team_id')['overall'].transform('sum')

In [102]:
#Crea columna nº de equipos por liga/año
df22['teams_in_league/year'] = df22.groupby('league_name')['club_team_id'].transform('nunique')

In [103]:
#Crea columna media liga/año
df22['mean_league/year'] = df22.groupby('league_name')['overall'].transform('mean')

In [104]:
#Crea columna media equipo/año
df22['mean_team/year'] = df22.groupby('club_team_id')['overall'].transform('mean')

In [105]:
#df solo con las columnas a transformar
posiciones=df22[["ls","st","rs","lw","lf","cf","rf","rw", 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'ldm', 'cdm', 'rdm',
     'lwb', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb',"gk"]]

In [106]:
#funcion para limpiar valores
def limp_posiciones(x):
    x.replace("+"," ")
    return x.split(" ")[0]

In [107]:
#limpiar en df
for i in posiciones.columns:
    df22[i]=df22[i].astype(str)
    df22[i]=df22[i].apply(limp_posiciones)

In [108]:
df22['lf'] = df22['lf'].str.replace(r'\+', ' ', regex=True)
df22['cf'] = df22['cf'].str.replace(r'\+', ' ', regex=True)
df22['rf'] = df22['rf'].str.replace(r'\+', ' ', regex=True)
df22['lam'] = df22['lam'].str.replace(r'\+', ' ', regex=True)
df22['ram'] = df22['ram'].str.replace(r'\+', ' ', regex=True)
df22['lm'] = df22['lm'].str.replace(r'\+', ' ', regex=True)
df22['rm'] = df22['rm'].str.replace(r'\+', ' ', regex=True)

df22['lf'] = df22['lf'].str.replace(r'\-', ' ', regex=True)
df22['cf'] = df22['cf'].str.replace(r'\-', ' ', regex=True)
df22['rf'] = df22['rf'].str.replace(r'\-', ' ', regex=True)
df22['lam'] = df22['lam'].str.replace(r'\-', ' ', regex=True)
df22['ram'] = df22['ram'].str.replace(r'\-', ' ', regex=True)
df22['lm'] = df22['lm'].str.replace(r'\-', ' ', regex=True)
df22['rm'] = df22['rm'].str.replace(r'\-', ' ', regex=True)


In [109]:
#funcion para limpiar valores
def limp_posiciones(x):
    return x.split(" ")[0]

In [110]:
for i in ['lf','cf','rf','lam','ram','lm','rm']:
    df22[i]=df22[i].astype(str)
    df22[i]=df22[i].apply(limp_posiciones)

In [125]:
X22=df22[list]

In [126]:
X22.fillna(0,inplace=True)

C:\Users\carlo\anaconda3\lib\site-packages\pandas\core\frame.py:4462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [127]:
y_pred=model_def.predict(X22)

In [131]:
ind_list=X22[y_pred==1].index.to_list()

In [132]:
#top 5 centrocampistas con mayor proyección
df22[df22.index.isin(ind_list)].sort_values(by='overall', ascending=False).head()

,sofifa_id,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,...,lb,lcb,cb,rcb,rb,gk,overall_team/year,teams_in_league/year,mean_league/year,mean_team/year
10886,255432,K. Salvatierra,Kevin Francisco Salvatierra Flores,centrocampista,65,80,1700000.0,500.0,19,2001-08-05,...,56+2,54+2,54+2,54+2,56+2,17+2,819.0,7.0,63.382716,63.000000
10974,259903,K. Sambonino,Kevin Josué Sambonino Terán,centrocampista,65,75,1500000.0,500.0,21,2000-02-14,...,62+2,62+2,62+2,62+2,62+2,17+2,563.0,7.0,64.337838,62.555556
10954,258923,Arribas,Sergio Arribas Calvo,centrocampista,65,81,1800000.0,16000.0,19,2001-09-30,...,47+2,42+2,42+2,42+2,47+2,16+2,1338.0,20.0,73.703846,78.705882
10955,259043,M. Samoise,Matisse Samoise,centrocampista,65,79,1700000.0,3000.0,19,2001-11-21,...,54+2,47+2,47+2,47+2,54+2,15+2,899.0,18.0,67.463918,69.153846
12041,257075,P. Nebel,Paul Nebel,centrocampista,64,83,1500000.0,2000.0,18,2002-10-10,...,44+2,37+2,37+2,37+2,44+2,13+2,923.0,18.0,71.148472,65.928571
